In [21]:
from selenium import webdriver
from time import sleep
from webdriver_manager.chrome import ChromeDriverManager
import csv
import requests
import pandas as pd
from bs4 import BeautifulSoup


# driver = webdriver.Chrome(ChromeDriverManager().install())


url = "https://www.hltv.org/stats"
players = []
teams = []
fields = ['Player', 'Team', 'Maps', 'K-D Diff', 'K/D', 'Rating (1.0)'] 
fields2 = ['Maps', 'Won-Lost', 'Pistol Win %', 'Round 2 Conversion', 'Round 2 Break'] 

# name of csv file 
def writeToCSV(players):
    filename = "data.csv"

    # writing to csv file 
    with open(filename, 'w') as csvfile: 
        # creating a csv writer object 
        csvwriter = csv.writer(csvfile) 

        # writing the fields 
        csvwriter.writerow(fields) 

        # writing the data rows 
        csvwriter.writerows(players)


def writeToCSV2(teams):
    filename = "data2.csv"

    # writing to csv file 
    with open(filename, 'w') as csvfile: 
        # creating a csv writer object 
        csvwriter = csv.writer(csvfile) 

        # writing the fields 
        csvwriter.writerow(fields2) 

        # writing the data rows 
        csvwriter.writerows(teams)
    
    
def get_parsed_page(url):
    # This fixes a blocked by cloudflare error i've encountered
    headers = {
        "referer": "https://www.hltv.org/stats",
        "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
    }

    return BeautifulSoup(requests.get(url, headers=headers).text, "lxml")



def driving():
    driver = webdriver.Chrome(ChromeDriverManager().install())
    driver.get('https://www.hltv.org/events/5603/blast-premier-spring-showdown-2021')
    element = driver.find_element_by_class_name('team-name')
    print(element)    
        
def playerPerformanceByDate():
    startDate = input("Start date (yyyy-mm-dd, leave blank for 'All-time' results): ")
    endDate = input("End date (yyyy-mm-dd, leave blank for 'All-time' results): ")
    print("Map Selection. Choose one from the following: all, vertigo, dust2, mirage, overpass, nuke, inferno, cobblestone, cache, train.")
    mapName = input("Choose the map: ")
    
    dateConversion = url + "/players?startDate=" + startDate + "&endDate=" + endDate + "&maps=de_"+mapName.lower() + "&rankingFilter=Top30";
    print(dateConversion)
    soup = get_parsed_page(dateConversion)
    statTable = soup.tbody;
    for tr in statTable.find_all("tr"):
        newArray = []
        for td in tr.find_all('td'):
            newArray.append(td.get_text())
        players.append(newArray)
    writeToCSV(players)
   

def pistolRoundByDate():
    startDate = input("Start date (yyyy-mm-dd, leave blank for 'All-time' results): ")
    endDate = input("End date (yyyy-mm-dd, leave blank if you want 'All-time' results): ")
    print("Map Selection. Choose one from the following: all, vertigo, dust2, mirage, overpass, nuke, inferno, cobblestone, cache, train.")
    mapName = input("Choose the map: ")
    teamSide = input("Enter team ('TERRORIST or COUNTER_TERRORIST'):")
    pistol = url + "/teams/pistols?startDate=" + startDate + "&endDate=" + endDate + "&maps=de_"+mapName.lower()+"&side=" + teamSide.upper() + "&rankingFilter=Top30"; 
    print(pistol)
    soup = get_parsed_page(pistol)
    pistolTable = soup.tbody;

    for tr in pistolTable.find_all("tr"):
        newArray = []
        for td in tr.find_all('td'):
            newArray.append(td.get_text())
        teams.append(newArray)
    writeToCSV2(teams)

    
# Calling functions and creating our csv files dynamically    
playerPerformanceByDate()
pistolRoundByDate()
print('--------------------------------------------------------------------------------')
print('RUN THE 2 CELLS BELOW TO SEE THE "PLAYER DATAFRAME" AND "TEAM PISTOL DATAFRAME"')
print('--------------------------------------------------------------------------------')
    

Start date (yyyy-mm-dd, leave blank for 'All-time' results): 2019-01-01
End date (yyyy-mm-dd, leave blank for 'All-time' results): 2020-01-01
Map Selection. Choose one from the following: all, vertigo, dust2, mirage, overpass, nuke, inferno, cobblestone, cache, train.
Choose the map: mirage
https://www.hltv.org/stats/players?startDate=2019-01-01&endDate=2020-01-01&maps=de_mirage&rankingFilter=Top30
Start date (yyyy-mm-dd, leave blank for 'All-time' results): 2019-01-01
End date (yyyy-mm-dd, leave blank if you want 'All-time' results): 2020-01-01
Map Selection. Choose one from the following: all, vertigo, dust2, mirage, overpass, nuke, inferno, cobblestone, cache, train.
Choose the map: mirage
Enter team ('TERRORIST or COUNTER_TERRORIST'):counter_terrorist
https://www.hltv.org/stats/teams/pistols?startDate=2019-01-01&endDate=2020-01-01&maps=de_mirage&side=COUNTER_TERRORIST&rankingFilter=Top30
--------------------------------------------------------------------------------
RUN THE 2 CELL

In [25]:
#Player Performance dataframe
player_df = pd.read_csv('data.csv')
player_df = player_df.dropna(axis='columns', how ='all')
player_df.head(10)


,Player,Maps,K-D Diff,K/D,Rating (1.0)
0,huNter-,45,226,1.29,1.30
1,ZywOo,45,239,1.31,1.29
2,blameF,37,170,1.28,1.28
3,Jame,39,231,1.38,1.26
4,EliGE,32,130,1.23,1.25
5,xsepower,44,220,1.30,1.21
6,Brehze,33,115,1.22,1.21
7,NAF,32,95,1.18,1.18
8,sdy,29,98,1.19,1.18
9,frozen,37,132,1.22,1.17


In [26]:
# Team Pistol dataframe
team_pistol_df = pd.read_csv('data2.csv')
team_pistol_df.head(10)

,Maps,Won-Lost,Pistol Win %,Round 2 Conversion,Round 2 Break
Valiance,27,13 - 14,48.1%,84.6%,7.1%
Natus Vincere,24,11 - 13,45.8%,100.0%,30.8%
Liquid,32,15 - 17,46.9%,80.0%,17.6%
fnatic,38,20 - 18,52.6%,85.0%,5.6%
CR4ZY,18,11 - 7,61.1%,81.8%,14.3%
forZe,44,20 - 24,45.5%,70.0%,17.4%
mousesports,38,20 - 18,52.6%,70.0%,38.9%
Heroic,40,24 - 16,60.0%,87.5%,37.5%
FURIA,33,16 - 17,48.5%,68.8%,5.9%
AVANGAR,38,15 - 23,39.5%,80.0%,0.0%


In [24]:
# url = 'https://www.hltv.org/events/5553/esl-pro-league-season-13'
# def eventDetails():
#     soup = get_parsed_page(url)
#     groups = soup.find_all("div", {"class": "group"})
#     for group in groups:
#         for tr in group.find_all("tr"):
#             print(tr.text)
# eventDetails()